## Web scrapping using python

#### References
1. [Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
2. [Web Scraping using Python](https://www.datacamp.com/community/tutorials/web-scraping-using-python)

In [1]:
# $ python3 -m venv venv
# $ . ./venv/bin/activate

In [142]:
#Better
!pip install requests BeautifulSoup4 fire
!pip install lxml
!pip install bs4
!pip install html5lib

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1283 sha256=4aa6567a075f167f43397259ebd74b6e15e9ad8dd974d874d9190d5122f437b5
  Stored in directory: c:\users\ken\appdata\local\pip\cache\wheels\df\bb\53\ffd18097aff565fd3d64285910fabf60510830d60b49f5d933
Successfully built bs4


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

In [3]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_tag_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [13]:
res = get_tag_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa','h2')
df = pd.DataFrame(res)
df.head(10)

,0
0,100. Jeffrey Gettleman (@gettleman)
1,99. Africa24 Media (@a24media)
2,98. Scapegoat (@andiMakinana)
3,97. Africa Check (@AfricaCheck)
4,96. James Copnall (@JamesCopnall)
5,95. Online Africa (@oafrica)
6,94. Patrick Ngowi (@PatrickNgowi)
7,93. DOS African Affairs (@StateAfrica)
8,92. MoadowAJE (@Moadow)
9,91. Brendan Boyle (@BrendanSAfrica)


In [140]:

# cleaning the data
df1 = df[0].str.split('.',expand = True)
df1.head(10)
#header = [['position','name','username']]
#df2=pd.DataFrame(header)
#frames= [df2, df1]
#df3=pd.concat(frames)
#print(df3.head(10))
#df4 = df3[1].str.split('(',expand = True)
#df4.head(10)
df2 = df1[1].str.split('(',expand = True)
df2.head(10)
df3 = df2[1].str.split(')',expand = True)
df3.tail(20)
df4 = df3.dropna(axis=0, how='any')
df5 = df4[0].replace('@','', regex=True)
df5


0           gettleman
1            a24media
2        andiMakinana
3         AfricaCheck
4        JamesCopnall
5             oafrica
6        PatrickNgowi
7         StateAfrica
8              Moadow
9      BrendanSAfrica
10        CityTshwane
11           VISI_Mag
12    andBeyondSafari
13    ThisIsAfricaTIA
14             sarzss
15      TheEIU_Africa
16     InvestInAfrica
17        malonebarry
18     artsouthafrica
19         KahnMorbee
20        JamalMOsman
21           iamsuede
22      mikestopforth
23    equal_education
24        t_mcconnell
25          forbeesta
26    hurricanevaness
27      BBCKarenAllen
28           jaxpanik
29       thisisafrica
           ...       
69            art2gee
70      JendayiFrazer
71        PeterGreste
72          NDOCKenya
73      Mo_IbrahimFdn
74    ParliamentofRSA
75        SandtonCity
76      _AfricanUnion
77          gertjohan
78      SmithInAfrica
79           hartleyr
80            liveamp
81          SamsungSA
82        BobSkinstad
83        

**Extracting Twitter data**

In [178]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

import numpy as np
import re

In [7]:

consumer_key = 'OwaWIDBtZ6QOIFjcTH5i6tKPZ'
consumer_secret = 'zsSqmcWmysfz42VgTlw401KmLNAYI2tvEUTW2ISX5vHnyoRHx8'
access_token = '275506568-ASxyT5sfrVXM17QIlGL5j2Ws2Qv2PHIyH4Fc8U0M'
access_token_secret = 'TE8xLOJQFD8uZrqS8xq3akXKy1aOBkVC6uOQ6A52Cvnc4'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = API(auth)


for target in df5:
    print("Getting data for " + target)
    item = api.get_user(target)
    print("name: " + item.name)
    print("screen_name: " + item.screen_name)
    print("description: " + item.description)
    print("statuses_count: " + str(item.statuses_count))
    print("friends_count: " + str(item.friends_count))
    print("followers_count: " + str(item.followers_count))
    

Getting data for gettleman
name: Jeffrey Gettleman
screen_name: gettleman
description: South Asia bureau chief for the New York Times. Author of Love, Africa, a memoir. Winner of the Pulitzer Prize. Retweets do not equal endorsements.
statuses_count: 3766
friends_count: 37
followers_count: 25703
Getting data for a24media
name: A24 Media
screen_name: a24media
description: Africa 24 produces compelling content that makes people want to peek, or gawk, or listen up. Our video creates emotion. Our shows make you think.

@yebo_live
statuses_count: 16852
friends_count: 3060
followers_count: 31292
Getting data for andiMakinana
name: Scapegoat
screen_name: AndiMakinana
description: In pursuit of scoops. I do not write headlines. In an abusive relationship with the mighty and glorious Buccaneers @Orlando_Pirates. @MikeTyson is the greatest.
statuses_count: 142178
friends_count: 2837
followers_count: 101162
Getting data for AfricaCheck
name: Africa Check
screen_name: AfricaCheck
description: Afri

TweepError: [{'message': 'User not found.', 'code': 50}]

In [141]:
targets = pd.DataFrame()

for target in df5:
    try:
        
        print("Getting data for " + target)
        tweets = api.user_timeline(screen_name=target,count=100,include_rts=True)
        user_details = api.get_user(target)
        tweet_list = [tweet for tweet in tweets]
        counter = 0
        favcounter= 0
        #initializing dataframe to store the data
        #targets = pd.DataFrame(columns=['username','popularity score','reach score','relevance score'])
        
        for tweet in tweet_list:
            retweetcount = tweet.retweet_count
            favorite = tweet.favorite_count
            
            counter += retweetcount
            favcounter += favorite
            print(tweet.text)
            print('number of retweets ' , retweetcount)
            print('number of likes ' , favorite)
            popularity_score = counter + favcounter   
            
            
            
        followers_count = user_details.followers_count
        friends_count = user_details.friends_count
        print('number of followers ',followers_count )
        print('number of people',target,'follows', friends_count)
        print('popularity score',popularity_score)
        reachscore = followers_count + friends_count
        print('reach score ', reachscore)
        data = pd.DataFrame(data=[target], columns=['username'])
        data['popularity score'] = np.array([popularity_score])
        data['reach score'] = np.array([reachscore])
        
        targets=pd.concat([targets,data],ignore_index = True)
        
        #targets.append({'username':target, 'popularity score': popularity_score,'reach score':reachscore,'relevance score' : '0'},ignore_index = True)
        
    except tweepy.TweepError :
        pass


targets

Getting data for gettleman
RT @SavvyPriya: @gettleman @HariNYT It's a very charming article on India's response to coronavirus. How Indian culture has defined the res…
number of retweets  1
number of likes  0
RT @KetanDayal: How world's 2 biggest troops operates at 15,000 ft. A ground report based on inputs from Ladakhis, retd. officials of ITBP&amp;…
number of retweets  1
number of likes  0
The illustrious Bollywood Bachchan clan tests positive for Covid. How did they get it? https://t.co/zvtAjJPIrh
number of retweets  7
number of likes  26
Dan Eldon was killed on this date in 1992. He squeezed the hell out of every day. I was lucky to know him.… https://t.co/Lyf9pJCrG9
number of retweets  4
number of likes  19
RT @Chun_Gurung: “He’s like god,” said Shivendra Singh Dungarpur, a filmmaker who has worked with @SrBachchan. “I’ve never seen a star havi…
number of retweets  1
number of likes  0
RT @RLAnalytica: https://t.co/TRWynhS8eu

Questions have been raised about the preparedness of #

,username,popularity score,reach score
0,gettleman,147094,25739
1,a24media,352,34352
2,andiMakinana,19761,104018
3,AfricaCheck,950,72627
4,JamesCopnall,125987,27006
5,oafrica,1549,34179
6,PatrickNgowi,7747,22975
7,StateAfrica,1,8
8,Moadow,14261,52688
9,BrendanSAfrica,3355,28828


In [269]:
targetss=targets.sort_values(['popularity score','reach score'],ascending = [False,False])
ranked=targetss.head(10)
ranked.to_csv('influencer ranking.csv')

In [266]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = simple_get(url)

In [188]:
res = get_tag_elements(response, 'strong')
res


['Botswana',
 'Comoros',
 'Eswatini',
 'Lesotho',
 'Madagascar',
 'Malawi',
 'Mauritius',
 'Mozambique',
 'Namibia',
 'Seychelles',
 'South Africa',
 'Zambia',
 'Zimbabwe',
 'Djibouti',
 'Eritrea',
 'Ethiopia',
 'Kenya',
 'Rwanda',
 'Somalia',
 'South Sudan',
 'Sudan',
 'Tanzania',
 'Uganda',
 'Angola',
 'Burundi',
 'Cameroon',
 'Central African Republic',
 'Chad',
 'Congo-Brazzaville',
 'Democratic Republic of Congo',
 'Equatorial Guinea',
 'Gabon',
 'São Tomé\xa0and Príncipe',
 'Benin',
 'Burkina Faso',
 'Cabo Verde',
 'Côte d’Ivoire',
 'Gambia',
 'Ghana',
 'Guinea',
 'Guinea-Bissau',
 'Liberia',
 'Mali',
 'Mauritania',
 'Niger',
 'Nigeria',
 'Senegal',
 'Sierra Leone',
 'Togo']

In [176]:
res = get_tag_elements(response, search={'find_all':{'class_':'twitter-tweet'}})
soup = BeautifulSoup(str(res))
soup2=soup.get_text()
soup
#res = pd.DataFrame(soup)
#res.head(10)

findaing all of {'class_': 'twitter-tweet'}


[<p dir="ltr" lang="en">The Deputy Prime Minister Themba Masuku has today met representatives of the private sector and employees' unions to map a collaborative effort in the fight against <a href="https://twitter.com/hashtag/COVID19?src=hash&amp;ref_src=twsrc%5Etfw">#COVID19</a>. <a href="https://t.co/EIYNGOEKRN">pic.twitter.com/EIYNGOEKRN</a></p>, '— Eswatini Government (@EswatiniGovern1) ', <a href="https://twitter.com/EswatiniGovern1/status/1241038139889721346?ref_src=twsrc%5Etfw">March 20, 2020</a>, <p dir="ltr" lang="en">GUIDELINES FOR SCHOOLS IN <a href="https://twitter.com/hashtag/MALAWI?src=hash&amp;ref_src=twsrc%5Etfw">#MALAWI</a> ON THE PREVENTION AND MANAGEMENT OF <a href="https://twitter.com/hashtag/COVID19?src=hash&amp;ref_src=twsrc%5Etfw">#COVID19</a> <a href="https://twitter.com/hashtag/CORONAVIRUS?src=hash&amp;ref_src=twsrc%5Etfw">#CORONAVIRUS</a> <a href="https://t.co/PL9R4XvGV3">pic.twitter.com/PL9R4XvGV3</a></p>, '— Malawi Government (@MalawiGovt) ', <a href="https:

In [270]:
res1 = get_tag_elements(response, search={'find_all':{'class_':'twitter-tweet'}})


str_cells2=str(res1)
cleantext=BeautifulSoup(str_cells2).get_text()
data=[]
for item in cleantext.split("20,"):
    clean=re.compile(r'@(\w*)')
    clean2=re.search(clean,item)
    if clean2 is not None:
        data.append({'value': clean2.group(0) })
    
    
df=pd.DataFrame(data)
df.columns = ["African leaders"]
df.to_csv('leaders.csv')

findaing all of {'class_': 'twitter-tweet'}


In [262]:
res_gov = get_tag_elements(response, tag='blockquote')
for r in res_gov:
    split_data = r.split('— ',maxsplit=1)[1].rsplit('(',maxsplit=1)
    name = split_data[0].split(',')[0].strip()
    handle =  split_data[1].rsplit(')',maxsplit=1)[0]
    leader = pd.DataFrame(data=[name], columns=['name'])
    data = []
    data.append(name)
    df = pd.DataFrame(data)
df

,0
0,Ministère de la Santé et de l'hygiène Publique
